In [196]:
from sklearn.base import BaseEstimator
import matplotlib.pyplot as plt

In [197]:
def h(x, i, theta):
    
    h = theta[0]
    
    for j in range(1, len(theta)):
        h += x[i][j-1] * theta[j]
    
    return h

In [198]:
ALFA = 0.0001

def gradiente_descendente(x,y,theta):
    ite = 0
    temp = [0] * len(theta)    
    
    m = len(x)
    n = len(theta)
    
    max_diff = 10
    
    while max_diff > 0.001:
        for i in range(m):
            
            custo = 0
            for j in range(n):
                custo += (h(x, i, theta) - y[i]) 
            
            temp[0] = theta[0] - ALFA/m * custo
            
            for j in range(1, n):
                temp[j] = theta[j] - ALFA/m * custo * x[i][j-1]   

            max_diff = min(abs(temp[i] - theta[i]) for i in range(len(theta)))
        
            for i in range(n):
                theta[i] = temp[i]
        ite += 1
    print(ite)

In [199]:
class LMSTrainer(BaseEstimator):
    
    def __init__(self, analitic=False):
            
        self.analitic = analitic
        self._trained = False
            
    def fit(self, X, theta,y=None):
        if self.analitic:
            
            # TODO: FAZER POR MATRIZES
            pass
        else:
            gradiente_descendente(X,y, theta)
        
        self._trained = True
        
        return self
        
    def predict(self, X, y=None):
        
        if not self._trained:
            raise RuntimeError("You must train classifer before predicting data!")
        
        return 1# resultado   

In [200]:
# 10 50 30 90 40 100
x = [[156,2,2,2,2,4.3],
     [135,3,2,2,1,3.4],
     [65,1,1,1,1,4.8],
     [100,2,2,1,2,4.2],
     [110,2,2,2,1,3.9],
     [75,2,1,2,2,4.6],
     [55,1,1,1,1,4.4]]

y = [2417,2130,1345,1784,1840,1610,1235]
theta = [10,50,30,90,40,100]
# theta = [0,0,0,0,0,0,0]

x2 = [[54,1,1,2,4,2.0],
      [455,8,9,20,5,5],
      [10,1,0,0,0,1],
      [101,2,4,1,1,3],
      [70,2,4,0,0,2.3],
      [47,2,1,1,1,4],
      [87,3,2,2,2,4.5]]


# x = [[2],
#     [2.4],
#     [1.5],
#     [3.5],
#     [3.5],
#     [3.7],
#     [3.7],
#     [2.4]]

# theta = [0,0]


# y = [196, 221, 136, 255, 244, 232, 267, 212]

trainer = LMSTrainer()
predictor = trainer.fit(x,theta,y)

teste = [0] * len(y)
for i in range(len(x)):
    teste[i] = h(x, i, theta)
    
print(teste, theta)

128977
[2419.7518849101457, 2127.2091594560743, 1330.8666926374633, 1774.201674303349, 1841.9695725437032, 1616.7522220765863, 1226.3635661035753] [513.6896293528888, 10.450312653388776, 23.981770577651776, -43.47566209093166, 60.33270201702496, 97.06793031055872]
